In [28]:
import pandas as pd
import numpy as np
df=pd.read_csv('prices.csv')

In [29]:
df

,ticker,date,open,high,low,close
0,AXISBANK,2010-01-04,198.779999,199.990005,197.619995,140.712021
1,AXISBANK,2010-01-05,199.600006,204.000000,199.139999,143.647934
2,AXISBANK,2010-01-06,204.000000,205.740005,198.509995,141.236801
3,AXISBANK,2010-01-07,199.289993,201.199997,197.820007,142.236725
4,AXISBANK,2010-01-08,200.600006,203.979996,199.270004,144.094727
...,...,...,...,...,...,...
47568,SRTRANSFIN,2020-06-09,636.900024,650.950012,613.000000,619.599976
47569,SRTRANSFIN,2020-06-10,622.599976,641.700012,612.099976,635.000000
47570,SRTRANSFIN,2020-06-11,615.250000,676.000000,615.250000,637.900024
47571,SRTRANSFIN,2020-06-12,615.000000,693.700012,606.000000,684.400024


In [ ]:
# def calculate_stochastic_oscillator(df, n=14):
    # Calculate the lowest low and highest high over the last 'n' periods
    low_min = df['low'].rolling(window=n).min()
    high_max = df['high'].rolling(window=n).max()

    # Calculate the Stochastic Oscillator %K and %D
    df['%K'] = 100 * (df['close'] - low_min) / (high_max - low_min)
    df['%D'] = df['%K'].rolling(window=3).mean()

    return df

def generate_stochastic_signals(df, overbought_threshold=80, oversold_threshold=20):
    # Generate buy and sell signals based on overbought and oversold regions
    df['Stochastic_Buy_Signal'] = np.where((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1)) & (df['%K'] < oversold_threshold), 1, 0)
    df['Stochastic_Sell_Signal'] = np.where((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1)) & (df['%K'] > overbought_threshold), -1, 0)

    return df

def calculate_bollinger_bands(df, n=20, k=2):
    # Calculate the rolling mean and standard deviation
    df['MA'] = df['close'].rolling(window=n).mean()
    df['Std'] = df['close'].rolling(window=n).std()

    # Calculate the upper and lower Bollinger Bands
    df['Upper_Band'] = df['MA'] + k * df['Std']
    df['Lower_Band'] = df['MA'] - k * df['Std']

    return df

def generate_bollinger_signals(df):
    # Generate buy and sell signals based on price crossing bands
    df['Bollinger_Buy_Signal'] = np.where((df['close'].shift(1) < df['Lower_Band'].shift(1)) & (df['close'] > df['Lower_Band']), 1, 0)
    df['Bollinger_Sell_Signal'] = np.where((df['close'].shift(1) > df['Upper_Band'].shift(1)) & (df['close'] < df['Upper_Band']), -1, 0)

    return df

def calculate_higherH_lowerL(df, window=20):
    df['HH_20'] = df['high'].rolling(window=window).max()
    df['LL_20'] = df['low'].rolling(window=window).min()
    
    return df
        
def calculate_valid_higher_highs(df):
    df['Valid_HH'] = df['high'] > df['HH_20'].shift(1)
    
    return df
        
def calculate_valid_lower_lows(df):
    df['Valid_LL'] = (df['low'] < df['LL_20'].shift(1)) & (df['low'] < df['low'].shift(1)) & (df['low'].shift(1) < df['LL_20'].shift(1))
    
    return df


def calculate_combined_signals(df):
    df['Buy_Signal'] = np.where((df['Stochastic_Buy_Signal'] == 1) & (df['Bollinger_Buy_Signal'] == 1) & (df['close'] < df['LL_20']), 1, 0)
    df['Sell_Signal'] = np.where((df['Stochastic_Sell_Signal'] == -1) & (df['Bollinger_Sell_Signal'] == -1) & (df['close'] > df['Valid_HH']), -1, 0)
    df['Returns'] = 0  # Initialize 'Returns' column with default value
    df['Percentage_Return'] = 0  # Initialize 'Percentage_Return' column with default value
    
    initial_capital = int(input())
    total_stocks = len(df['ticker'].unique())
    capital_per_stock = initial_capital / total_stocks
    temp_capital_per_stock = initial_capital / total_stocks
    df['capital_per_stock'] = capital_per_stock
    df['position'] = 'none' 
    df['portfolio_value'] = initial_capital
    flag = 0

    for i, row in df.iterrows():
        # Restoring the value of capital for each stock as the ticker symbol changes
        if i == 0 or row['ticker'] != df.loc[i - 1, 'ticker']:
            capital_per_stock = initial_capital / total_stocks

            
        if row['Buy_Signal'] == 1 and capital_per_stock != 0 and flag == 0:
            buy_price = row['close']
            num_shares_bought = capital_per_stock // buy_price
            capital_per_stock = 0
            df.at[i, 'position'] = 'buy' 
            df.at[i, 'portfolio_value'] -= num_shares_bought*buy_price 
            flag = i
            
        # Case for short selling
        if row['Buy_Signal'] == 1 and flag == -1:
            buy_price = row['close']
            final_cash_balance = capital_per_stock - (buy_price * num_shares_shorted)
            df.at[i, 'portfolio_value'] +=final_cash_balance
            returns = final_cash_balance - temp_capital_per_stock
            percentage_returns = (returns / temp_capital_per_stock) * 100
            df.at[i, 'Returns'] = returns
            df.at[i, 'Percentage_Return'] = percentage_returns  
            capital_per_stock = final_cash_balance
            flag = 0
            
        elif row['Buy_Signal'] == 1 and capital_per_stock == 0:
            df.at[i, 'position'] = 'hold'  
            continue
        
        elif row['Sell_Signal'] == -1:
            sell_price = row['close']
            if capital_per_stock == 0:
                earnings = sell_price * num_shares_bought
                returns = earnings - temp_capital_per_stock   
                percentage_returns = (returns / initial_capital) * 100
                df.at[i, 'Returns'] = returns
                df.at[i, 'Percentage_Return'] = percentage_returns  
                capital_per_stock = capital_per_stock + returns
                df.at[i, 'position'] = 'sell'
                df.loc[flag:i, 'position'] = 'hold'
                df.loc[flag:i-1, 'capital_per_stock'] = 0
                df.at[i, 'capital_per_stock'] = capital_per_stock
                df.at[i, 'portfolio_value']+= earnings
                flag = i
                
            # if the first signal we get is a sell signal, we start short selling
            elif capital_per_stock != 0:
                if flag == -1:
                    df.at[i, 'position'] = 'hold'  
                    continue
                else:
                    num_shares_shorted = capital_per_stock // sell_price
                    stock_proceeds = sell_price * num_shares_shorted
                    temp_capital_per_stock = capital_per_stock
                    capital_per_stock = capital_per_stock + stock_proceeds
                    flag = -1
                
        elif row['Sell_Signal'] == 0 and row['Buy_Signal'] == 0:
            continue
    
#     print(portfolio_value)
#     final_portfolio_value = portfolio_value
#     initial_portfolio_value = initial_capital
   
#     num_years = (pd.to_datetime(df['date']) - pd.to_datetime(df['date'].iloc[0])).dt.days / 365.25
#     df['num_years'] = num_years
#     print(num_years)

#     cagr = (final_portfolio_value / initial_portfolio_value) ** (1 / num_years) - 1

#     df['Drawdown'] = (df['Portfolio_Value'].cummax() - df['Portfolio_Value']) / df['Portfolio_Value'].cummax()
#     max_drawdown = data['Drawdown'].max()

   # df['CAGR'] = cagr
    #data['Max_Drawdown'] = max_drawdown
       

    return df

# Calculate Stochastic Oscillator
df = calculate_stochastic_oscillator(df)

# Generate Stochastic signals
df = generate_stochastic_signals(df)

# Calculate Bollinger Bands
df = calculate_bollinger_bands(df)

# Generate Bollinger signals
df = generate_bollinger_signals(df)

df=calculate_higherH_lowerL(df)

df=calculate_valid_higher_highs(df)

df= calculate_valid_lower_lows(df)

# Calculate combined signals
df = calculate_combined_signals(df)


print(df[['date', 'close', 'MA', 'Upper_Band', 'Lower_Band', 'Stochastic_Buy_Signal', 'Stochastic_Sell_Signal', 'Bollinger_Buy_Signal', 'Bollinger_Sell_Signal', 'Buy_Signal', 'Sell_Signal', 'Returns', 'Percentage_Return', 'position']])

In [31]:
def calculate_cagr(data):
    final_portfolio_value = data['portfolio_value']
    initial_portfolio_value = data['portfolio_value'].iloc[0]
    # num_years = pd.to_datetime(prices['date']) - pd.to_datetime(prices['date'].loc[0])
    # num_years = num_years/np.timedelta64(1, 'Y')
    num_years = (pd.to_datetime(data['date']) - pd.to_datetime(data['date'].iloc[0])).dt.days / 365.25
    data['num_years'] = num_years

    cagr = (final_portfolio_value / initial_portfolio_value) ** (1 / num_years) - 1
    
    data['CAGR'] = cagr
    
    return data    

In [36]:
df=calculate_cagr(df)
df
filtered_data = df[df['CAGR'] != 0]
print(filtered_data)

           ticker        date         open         high          low  \
234      AXISBANK  2010-12-09   252.479996   259.799988   245.940002   
1609     AXISBANK  2016-07-13   563.700012   563.700012   551.099976   
3036   BAJAJFINSV  2011-11-17   495.112000   504.985992   489.364990   
3039   BAJAJFINSV  2011-11-22   473.056000   479.147003   469.665985   
3095   BAJAJFINSV  2012-02-10   500.074005   509.358002   491.772003   
5378   BAJAJHLDNG  2010-12-10   820.500000   850.000000   786.750000   
5552   BAJAJHLDNG  2011-08-22   720.000000   748.000000   716.400024   
7460   BAJAJHLDNG  2019-05-29  3513.750000  3521.949951  3469.000000   
11591  BANKBARODA  2013-01-31   166.210007   174.389999   166.210007   
11825  BANKBARODA  2014-01-13   123.180000   124.699997   121.290001   
12521  BANKBARODA  2016-11-17   174.800003   179.600006   172.649994   
13710       GICRE  2019-01-24   248.449997   256.000000   247.699997   
14138        HDFC  2010-05-20   537.739990   545.000000   529.28

In [37]:
# Calculating drawdown percentage 
def calculate_drawdown(data):

    data['Peak'] = data['portfolio_value'].cummax()
    data['Drawdown'] = ((data['Peak'] - data['portfolio_value']) / data['Peak'])

    return data

In [39]:
df=calculate_drawdown(df)
max_drawdown = df.groupby('ticker')['Drawdown'].max()
print(max_drawdown)

ticker
AXISBANK      0.105874
BAJAJFINSV    0.137189
BAJAJHLDNG    0.411972
BAJFINANCE    0.411972
BANDHANBNK    0.411972
BANKBARODA    0.432737
GICRE         0.411972
HDFC          0.427991
HDFCAMC       0.411972
HDFCBANK      0.433948
HDFCLIFE      0.411972
IBULHSGFIN    0.414171
ICICIBANK     0.411972
ICICIGI       0.411972
ICICIPRULI    0.412409
INDUSINDBK    0.448904
KOTAKBANK     0.415454
L&TFH         0.436050
NIACL         0.411972
PEL           0.540773
PFC           0.411972
PNB           0.411972
SBILIFE       0.411972
SBIN          0.421457
SRTRANSFIN    0.443910
Name: Drawdown, dtype: float64


In [40]:
total_trades = len(df[df['Buy_Signal']!=0]) + len(df[df['Sell_Signal']!=0])
winning_trades = len(df[df['Percentage_Return'] > 0])
win_probability = winning_trades / total_trades * 100
losing_trades= total_trades-winning_trades
win_loss_ratio= winning_trades/losing_trades

print(f"Total trades: {total_trades}")
print(f"Winning trades: {winning_trades}")
print(f"Win probability: {win_probability:.2f}%")
print(f"Win:Loss: {win_loss_ratio}")

Total trades: 200
Winning trades: 7
Win probability: 3.50%
Win:Loss: 0.03626943005181347


In [45]:
max_profit=df['Percentage_Return'].max()
max_loss=df['Percentage_Return'].min()
print(f"Max profit: {max_profit} ")
print(f"Max loss: {max_loss} ")

Max profit: 219.02243934711692 
Max loss: -890.6953023122455 
